In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

TensorFlow version: 2.6.0
Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
epochs = 12

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


class BasicCNN(Model):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.conv2 = Conv2D(64, 3, activation='relu')
        self.maxp1 = MaxPool2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxp1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


model = BasicCNN()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


# Use tf.GradientTape to train the model:
@tf.function
def train(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    
@tf.function
def test(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_fn(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)
    

Metal device set to: Apple M1 Max


2021-10-28 08:27:52.084084: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-10-28 08:27:52.084201: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
##### SMALLER BATCH SIZE ######

batch_size = 32

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [4]:
%%time

for epoch in range(epochs):
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train(images, labels)

    for test_images, test_labels in test_ds:
        test(test_images, test_labels)

    print(
    f'Epoch {epoch+1}\n------------------------------- \n'
    f'Test loss: {test_loss.result()}, Test acc: {test_accuracy.result()*100:.2f}% \n'
  )

2021-10-28 08:27:52.340008: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-28 08:27:52.340653: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-10-28 08:27:52.340699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-28 08:27:58.743700: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-28 08:28:00.254984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1
------------------------------- 
Test loss: 0.04670846089720726, Test acc: 98.57% 

Epoch 2
------------------------------- 
Test loss: 0.036258287727832794, Test acc: 98.79% 

Epoch 3
------------------------------- 
Test loss: 0.03798925131559372, Test acc: 98.86% 

Epoch 4
------------------------------- 
Test loss: 0.03681561350822449, Test acc: 98.82% 

Epoch 5
------------------------------- 
Test loss: 0.038603901863098145, Test acc: 98.97% 

Epoch 6
------------------------------- 
Test loss: 0.03827790543437004, Test acc: 99.06% 

Epoch 7
------------------------------- 
Test loss: 0.03957931697368622, Test acc: 99.05% 

Epoch 8
------------------------------- 
Test loss: 0.04967956244945526, Test acc: 99.03% 

Epoch 9
------------------------------- 
Test loss: 0.04598122835159302, Test acc: 99.09% 

Epoch 10
------------------------------- 
Test loss: 0.06989841908216476, Test acc: 98.75% 

Epoch 11
------------------------------- 
Test loss: 0.060616690665483475, Te

In [5]:
##### BIGGER BATCH SIZE ######
#### simulating larger images with higher memory reqs

batch_size = 4096

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [6]:
%%time

for epoch in range(epochs):
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train(images, labels)

    for test_images, test_labels in test_ds:
        test(test_images, test_labels)

    print(
    f'Epoch {epoch+1}\n------------------------------- \n'
    f'Test loss: {test_loss.result()}, Test acc: {test_accuracy.result()*100:.2f}% \n'
  )

2021-10-28 08:29:27.573373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-28 08:29:29.091203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-28 08:29:29.215053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-28 08:29:29.415218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1
------------------------------- 
Test loss: 0.0502527579665184, Test acc: 99.02% 

Epoch 2
------------------------------- 
Test loss: 0.04913012310862541, Test acc: 99.07% 

Epoch 3
------------------------------- 
Test loss: 0.0485159307718277, Test acc: 99.05% 

Epoch 4
------------------------------- 
Test loss: 0.048133011907339096, Test acc: 99.05% 

Epoch 5
------------------------------- 
Test loss: 0.047891538590192795, Test acc: 99.05% 

Epoch 6
------------------------------- 
Test loss: 0.047745075076818466, Test acc: 99.05% 

Epoch 7
------------------------------- 
Test loss: 0.047654639929533005, Test acc: 99.05% 

Epoch 8
------------------------------- 
Test loss: 0.04757767543196678, Test acc: 99.05% 

Epoch 9
------------------------------- 
Test loss: 0.04752976819872856, Test acc: 99.06% 

Epoch 10
------------------------------- 
Test loss: 0.0475139245390892, Test acc: 99.07% 

Epoch 11
------------------------------- 
Test loss: 0.047478847205638885, Tes